### Porque utilizar um modelo de regressão logística?

Um modelo de regressão logística é ótimo para previsões relacionadas a categorias, no caso desse exercício se o passageiro(a) sobreviveu ou não com base em suas características

### Dataprep

#### Ler o arquivo fonte

In [52]:
import pandas as pd

## ler o excel bruto
df = pd.read_csv('./titanic_v2.csv')

print(df.dtypes)

df.groupby(['Survived', 'Sex']).count()

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object


Pclass  Name  Age  Siblings/Spouses Aboard  \
Survived Sex                                                  
0        female      81    81   81                       81   
         male       464   464  464                      464   
1        female     233   233  233                      233   
         male       109   109  109                      109   

                 Parents/Children Aboard  Fare  
Survived Sex                                    
0        female                       81    81  
         male                        464   464  
1        female                      233   233  
         male                        109   109

#### Lidar com nulos

In [24]:
df.apply(lambda x : sum(x.isnull()), axis=0)


Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64

Não há nulos

#### Remover colunas com valores únicos ou irrelevantes

In [25]:
df = df.drop(axis=1, columns=['Name']) # o nome da pessoa não tem correlação com caso ela tenha sobrevivido ou não

#### Transformar categorias em números

In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in df.columns:
    df[i] = le.fit_transform(df[i])

### Treinamento do modelo

#### Separando os dados

In [27]:
df_vars_indep = df.drop('Survived', axis=1)

df_alvo = df[['Survived']]

from sklearn.model_selection import train_test_split

vars_train, vars_test, res_train, res_test = train_test_split(df_vars_indep, df_alvo, test_size=0.3, random_state=777)

#### Treinando o modelo

In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000).fit(vars_train, res_train)

res_prev = model.predict(vars_train) # previsões geradas

res_prev

/home/ryguigas/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,

#### Medindo performance do modelo

##### Usando dados de treino

In [54]:
from sklearn.metrics import accuracy_score, confusion_matrix

predict_train = model.predict(vars_train)

print(accuracy_score(res_train, predict_train)) # percentual de acerto
print(confusion_matrix(res_train, predict_train, labels=[0, 1])) # matriz de confusão (y é a fonte e x é a previsão, a ordem dos valores segue do)

# 330 -> treino 0, modelo 0
# 53 -> treino 1, modelo 0
# 69 -> treino 0, modelo 1
# 165 -> treino 1, modelo 1

0.8032258064516129
[[333  53]
 [ 69 165]]


Utilizando os dados de treino, o modelo teve um quantidade baixa de erros (20%) e a matriz de confusão aponta que o modelo consegue prever corretamente boa parte das previsões e a diagonal do erro indica que o modelo tem uma leve tendência a ser otimista e considerar que uma pessoa não morreu

##### Usando dados de teste

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix

predict_test = model.predict(vars_test)

print(accuracy_score(res_test, predict_test)) # percentual de acerto
print(confusion_matrix(res_test, predict_test)) # matriz de confusão 

0.8014981273408239
[[137  22]
 [ 31  77]]


Utilizando os dados de teste, o modelo manteve quase a mesma previsão, indicando que não teve overfitting ou underfitting debilitantes ao modelo

### Conculsão

O modelo pode ser usado em produção por ter atingido uma precisão alta (80%) e não demostrar underfitting, demonstrado por errar bastante as previsões, ou overfitting, demonstrado por uma precisão altíssima do modelo.